In [1]:
%matplotlib inline 

from pandas import Series 
import pandas as pd 
import numpy as np
from IPython.display import HTML
from IPython.display import Image
import matplotlib.pyplot as plt
from io import BytesIO
import re

#data = pd.read_csv('1.C.1.csv', sep=';', index_col=[0, 1, 2, 3])
#data.sort_index(inplace=True)

#idx = pd.IndexSlice
#tmp = data.loc[idx[:, 'Psychologie', :, 2018], :]

#print(tmp.index.get_level_values('university').unique())
#display(data.loc['FWF'].xs(2018, level='year'))
#display(data.xs(2018, level='year'))

university_short_forms = { 'Universität Wien': 'UWI',
                           'Universität Graz': 'UGR',
                           'Universität Innsbruck': 'UIN',
                           'Universität Salzburg': 'USA',
                           'Universität Klagenfurt': 'UKL',
                           'Universität für Bodenkultur Wien': 'BKW',
                           'Universität für künstlerische und industrielle Gestaltung Linz': 'UGL',
                           'Universität für angewandte Kunst Wien': 'UAW',
                           'Universität Mozarteum Salzburg': 'UMS',
                           'Medizinische Universität Wien': 'MUW',
                           'Medizinische Universität Graz': 'MUG',
                           'Medizinische Universität Innsbruck': 'MUI',
                           'Universität Linz': 'ULI',
                           'Akademie der bildenden Künste Wien': 'ABW',
                           'Technische Universität Wien': 'TUW',
                           'Technische Universität Graz': 'TUG',
                           'Veterinärmedizinische Universität Wien': 'VMW',
                           'Montanuniversität Leoben': 'MUL',
                           'Universität für Weiterbildung Krems': 'UWK',
                           'Wirtschaftsuniversität Wien': 'WUW',
                           'Universität für Musik und darstellende Kunst Wien': 'MKW',
                           'Universität für Musik und darstellende Kunst Graz': 'MKG'
                         }

def shorten(name):
    if name in university_short_forms:
        return university_short_forms[name]
    else:
        return name


class Dimension:
    @classmethod
    def create(cls, dataframe, dimensions):
        dims = dict()
        for dim in dimensions:
            assert dim in dataframe.index.names, "There is no index '" + dim + "'"
            cats = dataframe.index.get_level_values(dim).unique()
            dims[dim] = Dimension(dim, cats.values)
        return dims
    
    def __init__(self, name, categories):
        self.__name = name
        self.__categories = categories   
    @property
    def name (self):
        return self.__name
    @property
    def categories(self):
        return self.__categories  
    def __str__(self):
        return self.name
        
class Datasheet:
    def __init__(self, filename, dimensions, timepoint_name, university_name, value_name):
        dataframe = pd.read_csv(filename, sep=';', index_col=list(range(len(dimensions)+2)))        
        dataframe.columns = [value_name]
        dataframe.sort_index(inplace=True)

        assert timepoint_name in dataframe.index.names, "There is no index '" + timepoint_name + "'"
        assert university_name in dataframe.index.names, "There is no index '" + university_name + "'"
        assert value_name in dataframe, "There is no colum with valid data called '" + value_name + "'"
        
        self.__dataframe = dataframe
        self.__timepoint_idx = timepoint_name
        self.__university_idx = university_name
        self.__value_column = value_name       
        self.__dimensions = Dimension.create(dataframe, dimensions + [university_name, timepoint_name])
        self.__timepoints = dataframe.index.get_level_values(timepoint_name).unique()
        self.__universities = dataframe.index.get_level_values(university_name).unique()
        self.__recent_timepoint = self.__timepoints.max()
                
    @property
    def value_column(self):
        return self.__value_column
    @property
    def university_idx(self):
        return self.__university_idx
    @property
    def timepoint_idx(self):
        return self.__timepoint_idx
    @property
    def dimensions(self):
        return self.__dimensions
    @property
    def most_recent(self):
        return self.__recent_timepoint
    @property
    def universities(self):
        return self.__universities
    
    def only(self, conditions):
        df = self.__dataframe
        if conditions is None:
            return df
        slices = list()
        for dim in self.dimensions.keys():
            if self.dimensions[dim].name in conditions.keys():
                slices.append(conditions[self.dimensions[dim].name])
            else:
                slices.append(slice(None))
        return df.loc[tuple(slices), :]
        
    def analyze(self, description, about, filtering=None):
        return Analysis(self, description, self.dimensions[about], self.only(filtering) \
                            .sum(level = [self.dimensions[about].name, self.university_idx]))


class Analysis:
    def __init__(self, sheet, properties, about, table):
        self.__sheet = sheet
        self.__table = table.unstack(about.name, fill_value=0)
        self.__about = about
        self.__props = properties
        
        self.__sums = table.sum(axis='columns')
        self.__sums.name = sheet.value_column
        self.__totals = table.sum(level=[about.name])[sheet.value_column]
        self.__grand_total = table.sum()
        self.__grand_total.name = 'grand total'
        
        assert self.__grand_total[sheet.value_column] == self.__sums.sum()
        
        self.__structure = Structure(self)
        self.__ranking = Ranking(self)

               
    @property
    def sheet(self):
        return self.__sheet
    @property
    def table(self):
        return self.__table
    @property
    def horizontal_sums(self):
        return self.__sums
    @property
    def sums(self):
        return self.horizontal_sums
    @property
    def vertical_sums(self):
        return self.__totals
    @property
    def totals(self):
        return self.vertical_sums
    @property
    def dim(self):
        return self.__about
    @property
    def description(self):
        return self.__props['description']
    @property
    def about(self):
        return self.__props['about']
    @property
    def scale_description(self):
        return self.__props['scale_description']
    
    def scale(self, value):
        return self.__props['scale'](value)
    def scale_format(self, value):
        return self.__props['scale_format'].format(self.scale(value))
    def format(self, value):
        return self.__props['format'].format(value)
    @property
    def value_format(self):
        return self.__props['format']
    
    
    @property
    def grand_total(self):
        return self.__grand_total
    @property
    def structure(self):
        return self.__structure
    @property
    def ranking(self):
        return self.__ranking
    
    def values(self, university):
        tmp = self.table.loc[university]
        tmp.name = self.sheet.value_column
        return tmp

    
    
class Ranking:
    def __init__(self, analysis):
        self.__analysis = analysis
        self.__rankings = analysis.table.rank(axis='rows', method='max', ascending=False)
        
        tmp = analysis.horizontal_sums.sum(level=self.analysis.sheet.university_idx)
        self.__total_ranking = tmp.rank(method='max', ascending=False).sort_values()
        self.__total_ranking.name = 'rank'
        
    @property
    def analysis(self):
        return self.__analysis
    @property
    def table(self):
        return self.__rankings
    
    def of(self, university, category=None):
        categories = self.__analysis.dim.categories
        retval = None
        if category == None:
            retval = self.__total_ranking.loc[university]
            retval.name = 'ranking'
        else:
            if category in categories:
                retval = self.__rankings[self.analysis.sheet.value_column, category][university].astype(int)
            else:
                retval = self.__rankings.loc[university].astype(int)
        return retval

    def better_than(self, university, category=None):
        if category == None:
            return self.__total_ranking[self.__total_ranking < self.__total_ranking[university]]
        else:
            tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()
            return tmp[tmp[university] > tmp]
        
    def best(self, category, top=1):
        tmp = self.__rankings[self.analysis.sheet.value_column, category].sort_values()[0:top]
        return tmp
       
        
            

class Structure:
    def __init__(self, analysis):
        self.__analysis = analysis
        tmp = analysis.sums.sum(level=[analysis.sheet.university_idx])
        self.__table = analysis.table.divide(tmp, axis=0).droplevel(level=0, axis='columns')
        self.__table.name = analysis.sheet.value_column    
        
    @property
    def table(self):
        return self.__table
    @property
    def analysis(self):
        return self.__analysis
    
    def most_important_to(self, university, cutoff=0.9):
        u = self.table.loc[university].sort_values(ascending=False)
        u.name = 'relevance'
        cumsum = u.cumsum()
        cut = (cumsum > cutoff).idxmax()
        return u[:cut]

    def of(self, university):
        tmp = self.table.loc[university].sort_values(ascending=False)
        tmp.name = 'relevance'
        return tmp
    
    def similarities_to(self, university):
        mirror = self.table.swapaxes('columns', 'rows', copy=False)
        uni = mirror[university]
        tmp = pd.DataFrame(data=uni.values).reindex_like(mirror)
        tmp.loc[:, :] = uni[:, None]
        return mirror.combine(tmp, np.minimum)
        
    def most_similar_to(self, university):
        m = self.similarities_to(university)
        result = m.sum(axis='rows')
        result = result.sort_values(ascending=False)[1:]
        result.name = 'similarity'
        return result
        
        
def highlight_places(column, series):
    l = len(series.index)
    if series[column] == 1:
        return ['background-color: green']*l
    elif series[column] == 2 or series[column] == 3:
        return ['background-color: yellow']*l
    else:
        return ['background-color: white']*l
            
class University:
    def __init__(self, name):
        self.__name = name
        self.__analyses = list()
        self.__similarity_matrices = dict()
        self.__similars = dict()
        self.__similarity_filter = 1/2
        
    def __lt__(self, other):
        return self.name < other.name
        
    @property
    def name(self):
        return self.__name
    @property
    def short(self):
        return shorten(self.__name)
    @property
    def similarity_filter(self):
        return self.__similarity_filter
    @property
    def similars(self):
        return self.__similars
    
    
    def add_analysis(self, analysis):
        a = analysis
        self.__analyses.append(analysis)
        self.__similarity_matrices[a.description] = a.structure.most_similar_to(self.name)
        self.__similars[a.description] = list()
            
        similars = self.__similarity_matrices[a.description]
        for i, other in enumerate(similars.index.values):
            if similars[other] < self.similarity_filter:
                if i == 0:
                    break
            else:
                self.__similars[a.description].append(other)
        
    def describe(self):
        display(HTML('<h2 style="page-break-before: always;">' + self.name + '</h2>'))
        display(HTML('<h3>Vergleichbare Universitäten in '
                    + '- und '.join(self.similars.keys()) 
                    + '-Struktur</h3><ul>'))
            
        similars = None
        for similars_list in self.similars.values():
            if similars is None:
                similars = set(similars_list)
            else:
                similars = similars.intersection(set(similars_list))
                
        for s in similars:
            display(HTML('<li style="margin-bottom: 5px;">' + s + '</li>'))
        if len(similars) == 0:
            display(HTML('<li><strong>Keine.</strong> Die ' + self.name + ' ist stark spezialisiert.</li>'))
        display(HTML('</ul>'))
        
        for a in self.__analyses:
            display(HTML('<h4>Vergleichbar bei der ' + a.description + '-Struktur</h4>'))
            sim_strs = list()
            if len(self.similars[a.description]) == 0:
                sim_strs.append('Die ' + self.name + ' ist im Bereich ' + a.description + ' spezialisiert.<br/>'
                                + '<strong>Keine</strong> andere Universität ähnelt der '
                                + self.name + ' in der ' 
                                + a.description + '-Struktur in auffälliger Weise.')
            
            similars = self.__similarity_matrices[a.description]
            sm = a.structure.similarities_to(self.name)
            for i, other in enumerate(self.similars[a.description]):
                if i > 5:
                    break
                sim_str = '<strong>' + other \
                          +  '</strong> (' + '{:.1%}'.format(similars[i]) \
                          + ', wegen &auml;hnlicher interner Relevanz von '
                similarities = sm[other].sort_values(ascending=False)
                sim_str += ', '.join(similarities.head(3).index.values)
                sim_str += ')'
                sim_strs.append(sim_str)
            display(HTML('<p>' + ', '.join(sim_strs) + '</p>'))
            
        display(HTML('<p><small>Universitäten mit einer Ähnlichkeit von weniger als '
                    + '{:.0%}'.format(self.similarity_filter)
                    + ' werden nicht angezeigt.</small></p>'))
        
        for a in self.__analyses:
            display(HTML("<h3>" + a.description + "-Struktur der " + self.short + "</h3>"))
            cutoff = 2/3
            df = a.structure.most_important_to(self.name, cutoff)
            identity = df
            rest = 1 - identity.sum()
            display(HTML('Angezeigt werden ' + a.about + ', die zusammen '
                         + '{:.1%}'.format(identity.sum()) + ' der ' + a.description 
                         + ' an der ' + shorten(self.name) + ' ausmachen. '
                         + 'Alle anderen ' + a.about + ' tragen je weniger als '
                         + '{:.1%}'.format(min(rest, identity.min())) + ' bei. Die Spalte \"Ranking\" bezieht sich '
                         + ' auf den &ouml;sterreichweiten Vergleich der absoluten Werte pro ' + a.about))
            
            df = df[df > 0] \
                .dropna()   \
                .to_frame()

            df = df.join(a.ranking.of(self.name, 'all')) \
                   .join(a.values(self.name))
            
            best = pd.Series()
            better = pd.Series()
            best.name = 'best'
            better.name = 'better'
            plots = dict()
            
            def get_valid_filename(s):
                s = str(s).strip().replace(' ', '_')
                return re.sub(r'(?u)[^-\w.]', '', s)
            
            for cat in df.index.get_level_values(level=a.dim.name):
                best_uni = a.ranking.best(cat).index.values[0]
                if best_uni == self.name:
                    best_uni = '(diese)'
                else: 
                    best_uni += ' (' + a.scale_format(a.values(best_uni)[a.sheet.value_column, cat]) + ')'
                    
                better_than_me = a.ranking.better_than(self.name, cat)
                better_unis = ', '.join(better_than_me.index.values)
                
                my_rank = a.ranking.of(self.name, cat)
                ranks = pd.DataFrame()
                ranks.name = cat
                for top5 in a.ranking.best(cat, top=max(my_rank+1, 4)).index.values:
                    ranks.at[top5, a.sheet.value_column] = a.scale(a.values(top5)[a.sheet.value_column, cat])
                    
                figure = plt.figure(figsize=(6,4))                    
                ax = ranks.plot.bar(ax=plt.gca(), legend=False, width=1, fontsize=20)
                for i, bar in enumerate(ax.patches):
                    if i == my_rank-1:
                        bar.set_facecolor('#D85010')
                    else:
                        bar.set_facecolor('#C0C0C0')
                ax.set_ylabel(a.scale_description, fontsize=20)
                labels = ax.get_xticklabels()
                for label in labels:
                    label.set_text(shorten(label.get_text()))
                ax.set_xticklabels(labels)
                filename = get_valid_filename(self.short + '_'.join([cat, a.description]) + '.png')
                plt.tight_layout()
                plt.savefig(filename, format='png', transparent=True)
                plt.close()
                plots[cat] = filename
                
                if better_unis == '':
                    better_unis = '(keine)'
                    
                best.at[cat] = best_uni
                better.at[cat] = filename
                
            df = df.join(best, on=a.dim.name).join(better, on=a.dim.name)
            df = df.droplevel(0)
            
            
            df.index.name = a.about
            df.columns = ['Intern', 'Ranking', a.description, 'Beste Universität', 'Universität im Vergleich']
            df = df.sort_values('Intern', ascending=False)
            df['Universität im Vergleich'] = df.index.map(lambda fos: plots[fos])
            
            sel = df[['Intern', a.description, 'Ranking', 'Universität im Vergleich']]
            style = sel.style.format({'Intern': '{:.2%}', 'Ranking': '{:}. Platz',
                                     a.description: a.value_format})
                        
            def show_image(filename):
                return '<img src="' + filename + '", width=250, height=200/>'
                
            style = style.apply(lambda s: highlight_places('Ranking', s), axis=1) \
                         .format({'Universität im Vergleich': show_image})
            
            display(HTML(style.render()))
            
            
                

ds = Datasheet('3.B.1.csv', ['fos'], 'year', 'university', 'funding')
analysis = ds.analyze({'description': 'Publikationen',
                       'about': 'WZW',
                       'format': '{:} Publikationen',
                       'scale_description': 'Publikationen',
                       'scale': lambda x: x,
                       'scale_format': '{:} Publikationen'
                      }, "fos", {'year': ds.most_recent})


ds2 = Datasheet('1.C.1.csv', ['fos', 'funder'], 'year', 'university', 'funding')
analysis2 = ds2.analyze({'description': 'Drittmittel',
                       'about': 'WZW',
                       'format': '{:,} €',
                       'scale_description': 'Millionen Euro',
                       'scale': lambda x: x/1000000,
                       'scale_format': '{:} Mio. €'
                      }, "fos", {'year': ds.most_recent})

universities = list()

for uni_name in ds.universities:
    u = University(uni_name)
    u.add_analysis(analysis)
    u.add_analysis(analysis2)
    universities.append(u)
    
for u in sorted(universities):
    u.describe()

,Intern,Publikationen,Ranking,Universität im Vergleich
WZW,,,,
Kunstwissenschaften,23.48%,31 Publikationen,9. Platz,
Bildende Kunst,18.18%,24 Publikationen,1. Platz,
Medien- und Kommunikationswissenschaften,8.33%,11 Publikationen,5. Platz,
Soziologie,8.33%,11 Publikationen,12. Platz,
Film und Fernsehen,7.58%,10 Publikationen,1. Platz,
Chemie,6.06%,8 Publikationen,12. Platz,


,Intern,Drittmittel,Ranking,Universität im Vergleich
WZW,,,,
Kunstwissenschaften,36.19%,"587,593 €",3. Platz,
Pädagogik / Vermittlung (2),34.23%,"555,820 €",1. Platz,


,Intern,Publikationen,Ranking,Universität im Vergleich
WZW,,,,
Klinische Medizin,60.94%,1521 Publikationen,2. Platz,
"Medizinisch-theoretische Wissenschaften, Pharmazie",16.91%,422 Publikationen,2. Platz,


,Intern,Drittmittel,Ranking,Universität im Vergleich
WZW,,,,
"Medizinisch-theoretische Wissenschaften, Pharmazie",67.87%,"34,148,791 €",1. Platz,


,Intern,Publikationen,Ranking,Universität im Vergleich
WZW,,,,
Klinische Medizin,66.93%,939 Publikationen,3. Platz,


...

,Intern,Drittmittel,Ranking,Universität im Vergleich
WZW,,,,
Architektur,36.97%,"653,979 €",1. Platz,
Pädagogik / Vermittlung (2),22.81%,"403,483 €",2. Platz,
Mediengestaltung,14.56%,"257,533 €",1. Platz,


,Intern,Publikationen,Ranking,Universität im Vergleich
WZW,,,,
Veterinärmedizin,79.32%,890 Publikationen,1. Platz,


,Intern,Drittmittel,Ranking,Universität im Vergleich
WZW,,,,
Veterinärmedizin,66.86%,"11,561,660 €",1. Platz,


,Intern,Publikationen,Ranking,Universität im Vergleich
WZW,,,,
Wirtschaftswissenschaften,54.58%,852 Publikationen,1. Platz,
Rechtswissenschaften,30.43%,475 Publikationen,6. Platz,


,Intern,Drittmittel,Ranking,Universität im Vergleich
WZW,,,,
Wirtschaftswissenschaften,55.96%,"7,199,353 €",1. Platz,
Rechtswissenschaften,20.98%,"2,699,082 €",1. Platz,


In [2]:
%%html
<style>
div.input {
    display:none;
}
</style>